# Settings

## Imports

In [1]:
import sys,tweepy,csv,re, requests, json
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import pandas as pd
import numpy as np
import os.path, time, re


from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import string
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gustavo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import warnings
warnings.filterwarnings('ignore') # We can suppress the warnings

# Getting Twitter

In [4]:
#token
config = dotenv_values(".env")
bearer_token = config['BEARER_TOKEN']


#connections
auth = tweepy.OAuth2BearerHandler({bearer_token})
api = tweepy.API(auth)



def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def get_tweets(query,file_name):
    search_url = 'https://api.twitter.com/2/tweets/search/recent'
    #search_url = 'https://api.twitter.com/1.1/search/tweets.json?'

    file_name = f'{file_name}.bz2'
    
    if os.path.exists(file_name) == False: #First checking if database exists
        print(f'Getting tweets...')
        
        # Querying the API
        json_response = connect_to_endpoint(search_url, query)
        
        '''
        It's necessary to complement with next twitters
        '''
 
        tweets_dt = pd.DataFrame.from_dict(json_response['data'])
        tweets_dt.to_csv(file_name, index=False,compression='bz2')
        
    else:
        create_dt = time.strftime("%d/%m/%Y %H:%M:%S",time.strptime(time.ctime(os.path.getmtime(file_name))))
        print(f'Reading {file_name}, created at {create_dt}')
        tweets_dt = pd.read_csv(file_name)
        
    return tweets_dt

In [5]:
NoOfTerms = int(10)

query_params = {
    #'q' : 'beef OR milk place:ea679934779f45c7',
    'query': 'Irish Farmers agriculture -is:retweet',
    'max_results': f'{NoOfTerms}',
    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
}


In [6]:
tweets = get_tweets(query_params,'tweets')
tweets.sample()

Reading tweets.bz2, created at 18/12/2022 09:57:04


,id,in_reply_to_user_id,conversation_id,reply_settings,edit_history_tweet_ids,public_metrics,created_at,source,author_id,referenced_tweets,text,lang
6,1603354556380889089,2.589328e+09,1603033104960167936,everyone,['1603354556380889089'],"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-12-15T11:41:29.000Z,Twitter Web App,1113016486321405952,"[{'type': 'replied_to', 'id': '160303310496016...",@agriculture_ie @McConalogue @UKRinIRL @Gerask...,en


# Clean Your Text Data

## Worlds extractions

In [88]:
def clean_tweet(x, link, keyword, usernames):
    list_of_lists =[]
    if link == True:
        list_of_links = []
        words = x.split(' ')
        for word in words:
            if re.search('http', word):
                list_of_links.append(re.split("\W+",word.lower()))
        if len(list_of_links) > 0:
            list_of_lists.append(list_of_links[0])
    
    if keyword == True:
        list_of_keywords = []
        words = x.split()
        for word in words:
            if word.startswith('#'):
                list_of_keywords.append(word)
        if len(list_of_keywords) > 0:
            list_of_lists.append(list_of_keywords)
            
    if usernames == True:
        list_of_usernames = []
        words = x.split()
        for word in words:
            if word.startswith('@'):
                list_of_usernames.append(word.lower().replace('@',''))
        if len(list_of_usernames) > 0:
            list_of_lists.append(list_of_usernames)
    
    return  [item for sublist in list_of_lists for item in sublist]

        
# keyword extraction from tweets
def get_keywords(x):
    list_of_keywords = []
    words = x.split()
    for word in words:
        if word.startswith('#'):
            list_of_keywords.append(word)
    return list_of_keywords


In [89]:
list_of_lists = tweets['text'].apply(lambda tweet : clean_tweet(tweet, link = True, keyword = False, usernames = True))
rem_list = [item for sublist in list_of_lists for item in sublist]

tweets['text_c'] = tweets['text'].apply( lambda tweet : ' '.join([word for word in re.split("\W+",tweet) if word.lower() not in rem_list]))

In [90]:
tweets['keywords'] = tweets['text'].apply( lambda tweet : get_keywords(tweet) )

In [91]:
n = 1
print(tweets['text'][n], '\n')
print(tweets['text_c'][n])

Celebrating🌍Africa’s food and farmers | Joan Baxter 🍁🎤📻✍️📰📚🇨🇦 @joan_baxter
https://t.co/Kno3bOkfKK @Comhlamh @farmersjournal @AgriAware @MacranaFeirme @IrelandDSA @teagasc @Irish_Aid @IrishSchSusty @TCBotanicGarden @NBGGlasnevin @agriculture_ie @ecowas_agric @BiaAmachAnseo 

Celebrating Africa s food and farmers Joan Baxter


## PoterStemmer

In [11]:
# Store the stopwords into the object named as "stop_words"
stop_words = stopwords.words('english')

# Store the string.punctuation into an object punct
punct = string.punctuation

# Initialise an object using a method PorterStemmer
stemmer = PorterStemmer()

In [31]:
def stremming(df, text_col, name_new_col):
    # Store the column of the dataframe named as "text"
    X = df[text_col]
    cleaned_data=[]
    # For loop from first value to length(X), ^a-zA-Z means include small and capital case letters
    for i in range(len(X)):
        text = re.sub('[^a-zA-Z]', ' ', X.iloc[i])
        text = text.lower().split()
        text = [stemmer.stem(word) for word in text if (word not in stop_words) and (word not in punct)]
        text = ' '.join(text)
        df.loc[ i ,name_new_col] = text
    print('Stremmer done!')

In [32]:
stremming(tweets, 'text_c', 'text_ps')

Stremmer done!


In [33]:
print(tweets['text'][0], '\n')
print(tweets['text_ps'][0], '\n')

@MarkBehan4 @McConalogue @GeraskoLarysa @FAO @agriculture_ie @FAODG @fiannafailparty @OgraFiannaFail OK this I agree with, their duty firstly should be to irish farmers, plenty of them struggling. 

ok agre duti firstli irish farmer plenti struggl 



# Sentiment Analyzes

TextBlob is a Python (2 and 3) library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more. [link](https://textblob.readthedocs.io/en/dev/index.html)

In [14]:
for i in tweets.index:
    text = tweets.loc[i,'text_ps']
    tweets.loc[i, 'TextBlob'] = TextBlob(text).sentiment.polarity
    #print(TextBlob(text).sentiment.polarity)
    tweets.loc[i, 'Vader'] = SentimentIntensityAnalyzer().polarity_scores(text)['compound']
    #print(SentimentIntensityAnalyzer().polarity_scores(text)['compound'], '\n')
    
tweets.sample(2)

,id,in_reply_to_user_id,conversation_id,reply_settings,edit_history_tweet_ids,public_metrics,created_at,source,author_id,referenced_tweets,text,lang,text_c,keywords,text_ps,TextBlob,Vader
7,1603322521692307456,NaN,1603322521692307456,everyone,['1603322521692307456'],"{'retweet_count': 2, 'reply_count': 0, 'like_c...",2022-12-15T09:34:12.000Z,Twitter Web App,194513294,NaN,Interesting times ahead for the Irish #organic...,en,Interesting times ahead for the Irish organic ...,"[#organic, #organicfarming]",interest time ahead irish organ sector organic...,0.000000,0.4588
4,1603511870475964416,1.210185e+18,1603429755537092624,everyone,['1603511870475964416'],"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-12-15T22:06:36.000Z,Twitter for Android,4267513701,"[{'type': 'replied_to', 'id': '160350533130847...",@GameOnClimate Most farmers accept that change...,en,Most farmers accept that changes are needed bu...,[],farmer accept chang need agricultur alway evol...,0.053333,0.4588


In [15]:
def get_polarity(df, col):

    polarity = 0
    neutral = 0
    wpositive = 0
    positive = 0
    spositive = 0
    wnegative = 0
    negative = 0
    snegative = 0
     
    for t in df.index:
        
        v = df.loc[t, col]
        polarity += v  # adding up polarities to find the average later

        if (v == 0):  # adding reaction of how people are reacting to find average later
            neutral += 1
            desc = 'neutral'
        elif (v > 0 and v <= 0.3):
            wpositive += 1
            desc ='weak_positive'
        elif (v > 0.3 and v <= 0.6):
            positive += 1
            desc = 'positive'
        elif (v > 0.6 and v <= 1):
            spositive += 1
            desc = 'strong_positive'
        elif (v > -0.3 and v <= 0):
            wnegative += 1
            desc = 'weak_negative'
        elif (v > -0.6 and v <= -0.3):
            negative += 1
            desc = 'negative'
        elif (v > -1 and v <= -0.6):
            snegative += 1
            desc = 'strong_negative'
         
        df.loc[t, f'{col}_desc'] = desc
        

    return {'polarity_sum':polarity,
            'polarity_mean':(polarity / len(df)),
            'neutral':neutral,
            'strong_positive':spositive,
            'positive':positive,
            'weak_positive':wpositive,
            'weak_negative':wnegative,
            'negative':negative,
            'strong_negative':snegative}




In [16]:
get_polarity(tweets,'TextBlob')

{'polarity_sum': 0.9783333333333334,
 'polarity_mean': 0.09783333333333334,
 'neutral': 5,
 'strong_positive': 0,
 'positive': 1,
 'weak_positive': 4,
 'weak_negative': 0,
 'negative': 0,
 'strong_negative': 0}

In [17]:
get_polarity(tweets,'Vader')

{'polarity_sum': 3.9924999999999997,
 'polarity_mean': 0.39925,
 'neutral': 1,
 'strong_positive': 2,
 'positive': 5,
 'weak_positive': 1,
 'weak_negative': 1,
 'negative': 0,
 'strong_negative': 0}

In [18]:
tweets.sample()

,id,in_reply_to_user_id,conversation_id,reply_settings,edit_history_tweet_ids,public_metrics,created_at,source,author_id,referenced_tweets,text,lang,text_c,keywords,text_ps,TextBlob,Vader,TextBlob_desc,Vader_desc
6,1603354556380889089,2.589328e+09,1603033104960167936,everyone,['1603354556380889089'],"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-12-15T11:41:29.000Z,Twitter Web App,1113016486321405952,"[{'type': 'replied_to', 'id': '160303310496016...",@agriculture_ie @McConalogue @UKRinIRL @Gerask...,en,Great any chance you might do something for ir...,[],great chanc might someth irish farmer see cont...,0.4,0.6249,positive,strong_positive


In [34]:
##Checking Twitters

n = 9
print('Text Original:', tweets.loc[n, 'text'], '\n',
     '-------------------------------------------------------------------------------------------')
print('Text Clear:', tweets.loc[n, 'text_c'], '\n',
     '-------------------------------------------------------------------------------------------')

print('Text Steammed:', tweets.loc[n, 'text_ps'], '\n',
     '-------------------------------------------------------------------------------------------')

print('KeyWords:', tweets.loc[n, 'keywords'], '\n',
     '-------------------------------------------------------------------------------------------')

print('TextBlob: ',tweets.loc[n, 'TextBlob'], tweets.loc[n, 'TextBlob_desc'])
print('Vader: ', tweets.loc[n, 'Vader'], tweets.loc[n, 'Vader_desc'])

Text Original: Diarmuid is passionate about the success of Irish #agriculture. He brings a wealth of knowledge &amp; experience to his interactions with #farmers &amp; other #agricultural stakeholders. https://t.co/mP9CC4LXQB 
 -------------------------------------------------------------------------------------------
Text Clear: Diarmuid is passionate about the success of Irish agriculture He brings a wealth of knowledge amp experience to his interactions with farmers amp other agricultural stakeholders 
 -------------------------------------------------------------------------------------------
Text Steammed: diarmuid passion success irish agricultur bring wealth knowledg amp experi interact farmer amp agricultur stakehold 
 -------------------------------------------------------------------------------------------
KeyWords: ['#agriculture.', '#farmers', '#agricultural'] 
 -------------------------------------------------------------------------------------------
TextBlob:  0.15 weak

In [20]:
tweets.iloc[:, -6:]

,keywords,text_ps,TextBlob,Vader,TextBlob_desc,Vader_desc
0,[],ok agre duti firstli irish farmer plenti struggl,0.250000,0.2960,weak_positive,weak_positive
1,[],celebr africa food farmer joan baxter,0.000000,0.0000,neutral,neutral
2,[],irish farmer want cull herd put busi ukrainian...,0.000000,0.4588,neutral,positive
3,[],screw irish farmer,0.000000,-0.1027,neutral,weak_negative
4,[],farmer accept chang need agricultur alway evol...,0.053333,0.4588,weak_positive,positive
5,[],turbul year input price output return dairi fa...,0.000000,0.5859,neutral,positive
6,[],great chanc might someth irish farmer see cont...,0.400000,0.6249,positive,strong_positive
7,"[#organic, #organicfarming]",interest time ahead irish organ sector organic...,0.000000,0.4588,neutral,positive
8,[],irish farmer journal northern correspond one j...,0.125000,0.3400,weak_positive,positive
9,"[#agriculture., #farmers, #agricultural]",diarmuid passion success irish agricultur brin...,0.150000,0.8720,weak_positive,strong_positive


## Irony and Sarcasm

Minimizing the impact of irony and sarcasm through hashing tags

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
news = pd.read_csv('sarcasm_headlines.bz2')
news.sample()

,headline,is_sarcastic
23224,'make daddy die' whispered into build-a-bear,1


In [24]:
print(news.isnull().any(axis = 0))

headline        False
is_sarcastic    False
dtype: bool


In [35]:
stremming(news, 'headline', 'headline_ps')

Stremmer done!


In [43]:
news.sample(2)

,headline,is_sarcastic,headline_ps
24456,family now openly wondering when grandma will die,1,famili openli wonder grandma die
7320,hubble telescope desperately struggling to con...,1,hubbl telescop desper struggl contact nasa wit...


In [48]:
def create_features(df, col, max_feat):
    features = df[col]

    # vectorizing the data with maximum features
    tv = TfidfVectorizer(max_features = max_feat)
    features = list(features)
    features = tv.fit_transform(features).toarray()
    
    return features

In [60]:
features = create_features(news, 'headline_ps', 100)
labels = news['is_sarcastic']

In [61]:
# getting training and testing data
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .05, random_state = 0)

In [62]:
print('\nLinear Support Vector Classifier:')
lsvc = LinearSVC()
lsvc.fit(features_train, labels_train)
print('Train: ',lsvc.score(features_train, labels_train))
print('Test: ',lsvc.score(features_test, labels_test))


print('\nGaussuan Naive Bayes:')
gnb = GaussianNB()
gnb.fit(features_train, labels_train)
print('Train: ',gnb.score(features_train, labels_train))
print('Test: ',gnb.score(features_test, labels_test))


print('\nLogistic Regression:')
lr = LogisticRegression()
lr.fit(features_train, labels_train)
print('Train: ',lr.score(features_train, labels_train))
print('Test: ',lr.score(features_test, labels_test))


print('\nRandom Forest Classifier:')
rfc = RandomForestClassifier(n_estimators = 10, random_state = 0)
rfc.fit(features_train, labels_train)
print('Train: ',rfc.score(features_train, labels_train))
print('Test: ',rfc.score(features_test, labels_test))


Linear Support Vector Classifier:
Train:  0.6592834903243605
Test:  0.6594311377245509

Gaussuan Naive Bayes:
Train:  0.6001655302881015
Test:  0.6175149700598802

Logistic Regression:
Train:  0.6597564340046506
Test:  0.6579341317365269

Random Forest Classifier:
Train:  0.7264414929255508
Test:  0.6594311377245509


### Prev

In [63]:
prev = create_features(tweets, 'text_ps', 100)

In [66]:
tweets['is_sarcastic'] = lsvc.predict(prev)

In [68]:
tweets.sample(4)

,id,in_reply_to_user_id,conversation_id,reply_settings,edit_history_tweet_ids,public_metrics,created_at,source,author_id,referenced_tweets,text,lang,text_c,keywords,text_ps,TextBlob,Vader,TextBlob_desc,Vader_desc,is_sarcastic
1,1603760301194739712,NaN,1603760301194739712,everyone,['1603760301194739712'],"{'retweet_count': 2, 'reply_count': 0, 'like_c...",2022-12-16T14:33:46.000Z,Twitter Web App,1321217184459272193,NaN,Celebrating🌍Africa’s food and farmers | Joan B...,en,Celebrating Africa s food and farmers Joan Baxter,[],celebr africa food farmer joan baxter,0.0,0.0000,neutral,neutral,1
3,1603537716993757190,1.978979e+08,1603038191728365568,everyone,['1603537716993757190'],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-12-15T23:49:18.000Z,Twitter for Android,1114989143145230341,"[{'type': 'replied_to', 'id': '160303819172836...",@McConalogue @GeraskoLarysa @FAO @agriculture_...,en,Screw the Irish farmers,[],screw irish farmer,0.0,-0.1027,neutral,weak_negative,1
6,1603354556380889089,2.589328e+09,1603033104960167936,everyone,['1603354556380889089'],"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-12-15T11:41:29.000Z,Twitter Web App,1113016486321405952,"[{'type': 'replied_to', 'id': '160303310496016...",@agriculture_ie @McConalogue @UKRinIRL @Gerask...,en,Great any chance you might do something for ir...,[],great chanc might someth irish farmer see cont...,0.4,0.6249,positive,strong_positive,0
2,1603707005147152385,1.978979e+08,1603038191728365568,everyone,['1603707005147152385'],"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-12-16T11:02:00.000Z,Twitter for Android,1469965454865321985,"[{'type': 'replied_to', 'id': '160303819172836...",@McConalogue @GeraskoLarysa @FAO @agriculture_...,en,Irish farmers We want to cull the herd and put...,[],irish farmer want cull herd put busi ukrainian...,0.0,0.4588,neutral,positive,0
